In [ ]:
from xml.etree.ElementTree import iterparse
import pandas as pd
"""
Pand ID always follows the tag "{http://www.kadaster.nl/schemas/imbag/lvc/v20090901}bron". Multiple instances of the tag for
ID are the same but can refer to for example an adress or pand. Since I'm interested in the building for now I'll only use the
ID referring to the building.

More data can be found in the XML files. Only taking the necesarry.
"""
def parseVBO(file):
    rows = []
    d = {"gebruiksdoelVerblijfsobject" : None, "oppervlakteVerblijfsobject": None, "verblijfsobjectStatus" : None, "begindatumTijdvakGeldigheid" : None, "einddatumTijdvakGeldigheid" : None, 
        "identificatie" : None, "pos" : None}
    idcheck = False
    for i, (tag, elem) in enumerate(iterparse(file)):

        if elem.tag == "{http://www.kadaster.nl/schemas/imbag/lvc/v20090901}gebruiksdoelVerblijfsobject":
            
            rows.append(d)
            
            d = {"gebruiksdoelVerblijfsobject" : None, "oppervlakteVerblijfsobject": None, "verblijfsobjectStatus" : None, "begindatumTijdvakGeldigheid" : None, "einddatumTijdvakGeldigheid" : None, 
            "identificatie" : None, "pos" : None}

            d["gebruiksdoelVerblijfsobject"] = elem.text

        elif elem.tag == "{http://www.kadaster.nl/schemas/imbag/lvc/v20090901}oppervlakteVerblijfsobject":
            d["oppervlakteVerblijfsobject"] = elem.text

        elif elem.tag == "{http://www.kadaster.nl/schemas/imbag/lvc/v20090901}verblijfsobjectStatus":  
            d["verblijfsobjectStatus"] = elem.text

        elif elem.tag == "{http://www.kadaster.nl/schemas/imbag/imbag-types/v20090901}begindatumTijdvakGeldigheid":
            d["begindatumTijdvakGeldigheid"] = elem.text

        elif elem.tag == "{http://www.kadaster.nl/schemas/imbag/imbag-types/v20090901}einddatumTijdvakGeldigheid":
            d["einddatumTijdvakGeldigheid"] = elem.text

        elif elem.tag == "{http://www.kadaster.nl/schemas/imbag/lvc/v20090901}bron":
            idcheck = True

        elif elem.tag == "{http://www.kadaster.nl/schemas/imbag/lvc/v20090901}identificatie" and idcheck:
            d["identificatie"] = elem.text
            idcheck = False

        elif elem.tag == "{http://www.opengis.net/gml}pos":
            d["pos"] = elem.text
        
    df = pd.DataFrame.from_records(rows)
    df["begindatumTijdvakGeldigheid"] =  pd.to_datetime(df['begindatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    df["einddatumTijdvakGeldigheid"] =  pd.to_datetime(df['einddatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    df.drop(index=df.index[0], axis=0, inplace=True)
    return df

In [ ]:
df = pd.DataFrame()
import os
# assign directory
directory = '../Data/BAG/PAND'
# iterate over files in
# that directory
tot = len(os.listdir(directory))
c = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    c += 1
    if c % 20 == 0:
        print("{} % done".format(c / tot * 100))
    if os.path.isfile(f):
        df = df.append(parsePanden(f))
        
df.to_csv("panden.csv")